# Task 5D – Teil 3: Feature Engineering & Embedding-Generierung

**Umfang dieses Notebooks (Teil 3):**

1. Laden der vorbereiteten Äußerungsdaten aus Teil 2 (`train_utterances.parquet` und `dev_utterances.parquet`)
2. Initialisierung eines geeigneten Satz-Embedding-Modells (`sentence-transformers`)
3. Berechnung von numerischen Embeddings für jede Äußerung im Datensatz
4. Speicherung der Ergebnisse als `.npz`-Dateien (NumPy-Container) zur späteren Nutzung

> **Hinweis:** Dieses Notebook erzeugt die semantischen Repräsentationen der Transkripte und bildet damit die Grundlage für das Clustering in Teil 4.


## 1) Bibliotheken laden & Konfiguration vorbereiten

In diesem Abschnitt werden alle benötigten Bibliotheken eingebunden und zentrale Konfigurationsparameter aus der Datei `run_config.json` geladen.

Dazu zählen insbesondere:

* das Projektverzeichnis (`PROJECT_ROOT`)
* die Speicherorte der Embedding-Dateien (`train_embeddings.npz`, `dev_embeddings.npz`)
* vorbereitende Einstellungen für das Clustering

> Ziel ist es, eine konsistente Arbeitsumgebung zu schaffen, auf der alle folgenden Schritte aufbauen können.


In [1]:
!pip install -q sentence-transformers

In [2]:
# ---------------------------------------------
# 1) Bibliotheken laden & Konfiguration vorbereiten
# ---------------------------------------------
# Ziel dieses Blocks:
# - Projektpfad und zentrale Parameter aus run_config.json laden
# - Pfade zu vorbereiteten Parquet-Dateien definieren (aus Teil 2)

from pathlib import Path  # Für systemunabhängige Pfadangaben
import pandas as pd       # Für Datenmanipulation mit DataFrames
import json               # Für das Einlesen der Konfigurationsdatei

# Projektverzeichnis festlegen (wurde in Teil 1 angelegt)
PROJECT_ROOT = Path.home() / "AUVIS/task5D_ml_prototype"

# Pfad zur Konfigurationsdatei
cfg_path = PROJECT_ROOT / "run_config.json"

# Konfigurationsdatei laden (JSON)
with open(cfg_path, "r", encoding="utf-8") as f:
    cfg = json.load(f)

# Pfade zu den vorbereiteten Parquet-Dateien (aus Teil 2)
PREPARED_DIR   = PROJECT_ROOT / "data" / "prepared"
PARQUET_TRAIN  = PREPARED_DIR / "train_utterances.parquet"
PARQUET_DEV    = PREPARED_DIR / "dev_utterances.parquet"


## 2) Modell vorbereiten

In diesem Schritt wird ein vortrainiertes **englisches Sprachmodell** aus der Bibliothek `sentence-transformers` geladen. Dieses Modell wandelt die transkribierten Äußerungen in numerische Repräsentationen (sogenannte *Embeddings*) um.

Diese Embeddings dienen als Grundlage für weiterführende Verfahren wie **Clustering**, **Klassifikation** oder **Ähnlichkeitsanalysen**. Das Modell basiert auf einem Transformer-Encoder und ist dafür optimiert, semantisch ähnliche Texte in ähnliche Vektoren zu überführen.

> Hinweis: Für die CHiME-Task werden überwiegend englischsprachige Daten verarbeitet. Entsprechend wird ein **englisch fine-getuntes Base-Modell** (z. B. `all-mpnet-base-v2`) verwendet.


In [3]:
# ---------------------------------------------
# 2) Modell vorbereiten
# ---------------------------------------------
# Ziel dieses Blocks:
# - Ein vortrainiertes Sentence-Transformer-Modell laden
# - Das Modell dient zur Erzeugung von Satz-Embeddings (numerische Repräsentationen)
# - Diese Embeddings sind später Grundlage für Clustering/Klassifikation

from sentence_transformers import SentenceTransformer  # HF-kompatible Bibliothek für Satz-Embeddings

# Modellwahl:
# - "all-mpnet-base-v2" ist ein leistungsstarkes englisches Modell
# - Gute semantische Trennung auch bei kurzen Texten
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

# Modell laden (aus Hugging Face Hub)
model = SentenceTransformer(MODEL_NAME)

# Bestätigung
print("Modell erfolgreich geladen:", MODEL_NAME)


Modell erfolgreich geladen: sentence-transformers/all-mpnet-base-v2


## 3) Embeddings berechnen

In diesem Schritt werden die transkribierten Äußerungen aus dem vorbereiteten Parquet-Format geladen und mit dem geladenen `SentenceTransformer`-Modell in numerische Vektoren (Embeddings) umgewandelt.

Jede Äußerung erhält ein semantisches **Satz-Embedding**, das ihren Inhalt in einem mehrdimensionalen Vektorraum repräsentiert. Diese Embeddings sind die Grundlage für alle nachfolgenden Verfahren wie Clustering oder Klassifikation.

**Ziel:**

* Einlesen der vorbereiteten Daten (Train / Dev)
* Batchweise Berechnung der Embeddings mit `model.encode(...)`
* Rückgabe als DataFrame zur Weiterverarbeitung oder Persistierung

> Hinweis: Für größere Datenmengen kann optional GPU-Beschleunigung oder Batch-Size-Tuning verwendet werden.


In [4]:
# ---------------------------------------------
# 3) Embeddings berechnen
# ---------------------------------------------
# Ziel dieses Blocks:
# - Eingelesene Äußerungen aus den vorbereiteten Parquet-Dateien laden
# - Textspalte (Standard: "text") in semantische Embeddings umwandeln
# - Nutzung des geladenen SentenceTransformer-Modells (GPU optional)
# - Rückgabe als NumPy-Array (ein Vektor pro Äußerung)

def embed_dataframe(df, model, text_column="text", batch_size=32):
    """
    Berechnet Satz-Embeddings für eine DataFrame-Spalte mit Text.

    Parameter:
    - df: Pandas DataFrame mit Textspalte
    - model: SentenceTransformer-Modell
    - text_column: Name der Textspalte im DataFrame (Standard: "text")
    - batch_size: Anzahl der Texte pro Mini-Batch (je nach GPU-Speicher anpassbar)

    Rückgabe:
    - NumPy-Array mit einem Vektor pro Zeile im DataFrame
    """
    from tqdm import tqdm
    tqdm.pandas()  # Fortschrittsbalken aktivieren

    # Modellencode: Liste von Strings → Liste von Embeddings
    embeddings = model.encode(
        df[text_column].tolist(),
        batch_size=batch_size,
        show_progress_bar=True
    )
    return embeddings

# ----------------------------
# Daten laden & Embedding berechnen
# ----------------------------

# Immer die neuen Multi-Session-Dateien verwenden
PARQUET_TRAIN = PROJECT_ROOT / "data" / "prepared" / "train_utterances_multisession.parquet"
PARQUET_DEV   = PROJECT_ROOT / "data" / "prepared" / "dev_utterances_multisession.parquet"

df_train = pd.read_parquet(PARQUET_TRAIN)
df_dev   = pd.read_parquet(PARQUET_DEV)

print("Train-Daten:", df_train.shape, "Sessions:", df_train['session_id'].nunique())
print("Dev-Daten:  ", df_dev.shape,   "Sessions:", df_dev['session_id'].nunique())

# Embeddings erzeugen
emb_train = embed_dataframe(df_train, model)
emb_dev   = embed_dataframe(df_dev, model)


Train-Daten: (19076, 7) Sessions: 56
Dev-Daten:   (8561, 7) Sessions: 25


Batches:   0%|          | 0/597 [00:00<?, ?it/s]

Batches:   0%|          | 0/268 [00:00<?, ?it/s]

## 4) Embeddings speichern

Nach der erfolgreichen Berechnung der Satz-Embeddings für Trainings- und Entwicklungsdaten speichern wir die Ergebnisse für die spätere Weiterverarbeitung.

**Ziel:** Persistenz der Embeddings als `.npy`-Dateien (NumPy-Format) zur effizienten Wiederverwendung in Downstream-Schritten wie Clustering oder Klassifikation.
Optional könnten die Embeddings auch als `.parquet` oder `.hdf5` gespeichert werden – das `.npy`-Format bietet jedoch eine besonders einfache und performante Handhabung bei numerischen Arrays.


In [5]:
# ---------------------------------------------
# 4) Embeddings speichern
# ---------------------------------------------
# Ziel dieses Blocks:
# - Die zuvor berechneten Embeddings als komprimierte NumPy-Dateien sichern
# - Format: .npz (komprimiert, speichert mehrere Arrays in einer Datei)
# - Je Split (Train/Dev) werden:
#   - die eindeutigen Äußerungs-IDs ("utt_id") und
#   - die zugehörigen Embedding-Vektoren gespeichert

import numpy as np  # NumPy für effiziente numerische Arrays

# Zielverzeichnis für Embeddings definieren
out_dir = PROJECT_ROOT / "data" / "embeddings"
out_dir.mkdir(parents=True, exist_ok=True)  # Verzeichnis anlegen (falls nicht vorhanden)

# Speicherung: Train-Split
np.savez_compressed(
    out_dir / "train_embeddings.npz",        # Zielpfad
    utt_ids=df_train["utt_id"].astype(str).values,  # Äußerungs-IDs als Strings
    embeddings=emb_train                     # Embedding-Vektoren (z. B. 768-dim)
)

# Speicherung: Dev-Split
np.savez_compressed(
    out_dir / "dev_embeddings.npz",          # Zielpfad
    utt_ids=df_dev["utt_id"].astype(str).values,    # Äußerungs-IDs als Strings
    embeddings=emb_dev                       # Embedding-Vektoren
)

# Bestätigung der gespeicherten Dateien
print("Gespeichert:")
for f in out_dir.glob("*.npz"):
    print(" -", f.name)


Gespeichert:
 - train_embeddings.npz
 - dev_embeddings.npz


### ✅ Nächste Schritte (für Teil 4)

Fahre mit dem nächsten Notebook **Teil 4: Clustering der Äußerungen zu Gesprächseinheiten** fort, sobald die Embeddings erfolgreich gespeichert wurden:

* Extraktion semantischer Merkmale mittels vortrainierter Sentence-Embedding-Modelle (`sentence-transformers`)
* Speicherung der Äußerungs-Embeddings als `.npz`-Dateien (komprimierte Vektor-Repräsentationen)
* Optionale Vorbereitung weiterführender Merkmale (prosodisch, visuell, kontextuell)
* Ziel: Grundlage schaffen für sprecherübergreifendes Clustering in Teil 4
